In [1]:
import numpy as np,os,sys,glob
import matplotlib.pylab as plt
from ioMicro import *

In [2]:
from scipy import ndimage as nd
import os,sys,glob,numpy as np
def get_dic_fls(analysis_fls):
    dic_fls = {} 
    for fl in analysis_fls:
        ifov = get_ifov(fl)
        if ifov not in dic_fls: dic_fls[ifov] = []
        if '_Q' in os.path.basename(fl):
            dic_fls[ifov]+=[fl]
    return dic_fls

class post_analysis():
    def __init__(self,analysis_fls = r'Y:\Glass_MERFISH\CGBB_ChrX_01_31_2023_Analysis\*_C*_Xh_RNAs.pkl',force=False):
        self.save_folder = self.analysis_folder =os.path.dirname(analysis_fls)
        save_Afls = self.save_folder+os.sep+'analysis_fls.npy'
        if os.path.exists(save_Afls) and not force:
            self.analysis_fls = np.load(save_Afls)
        else:
            self.analysis_fls = np.array(glob.glob(analysis_fls))
            np.save(save_Afls,self.analysis_fls)
        self.dic_fls = get_dic_fls(self.analysis_fls)
        
    def get_Xh(ifov=0,iH=0,icol=0):
        self.iH = iH
        self.icol = icol
        self.ifov=ifov
        
        self.fit_fls = self.dic_fls[self.ifov]
        self.fit_fl = self.fit_fls[self.iH]
        self.Xhs,self.dic_drift = pickle.load(open(fl,'rb'))
        self.Xh = Xhs[self.icol]
    def load_segm(self,force=False):
        analysis_folder = self.save_folder
        self.fov= fov = get_fov(self.fl)
        segm_folder = analysis_folder+os.sep+'Segmentation'
        dapi_fls_fl = segm_folder+os.sep+'dapi_fls_fl.npy'
        if os.path.exists(dapi_fls_fl) or force:
            dapi_fls = glob.glob(segm_folder+os.sep+'*.npz')
            np.save(dapi_fls_fl,dapi_fls)
        else:
            dapi_fls = np.load(dapi_fls_fl)
        self.dapi_fls = dapi_fls
        self.fl_dapi = fl_dapi = [fl for fl in self.dapi_fls if fov in fl and self.set_ in get_H_fld(fl)][0]
        #save_fl  = segm_folder+os.sep+os.path.basename(fl_dapi).split('.')[0]+'--'+os.path.basename(os.path.dirname(fl_dapi))+'--dapi_segm.npz'
        self.im_segm,self.shape = np.load(fl_dapi)['segm'],np.load(fl_dapi)['shape']
        self.im_segmf = resize(self.im_segm,self.shape)
    def get_Xh_Q(self,ifov=0,Q = 'Q1'):
        if True:#try:
            self.ifov=ifov
            self.fit_fls = self.dic_fls[self.ifov]
            fls = self.fit_fls
            iQ = int(Q[1:])
            iQs = [list(eval(fl.split('Q')[-1].split('_')[0])) for fl in fls]
            self.iH,self.icol = [[iH_,iQ_.index(iQ)] for iH_,iQ_ in enumerate(iQs) if iQ in iQ_][0]
            self.fl = self.fit_fls[self.iH]
            if getattr(self,"current_fl","")!=self.fl:
                self.current_fl = self.fl
                self.Xhs,self.dic_drift = pickle.load(open(self.fl,'rb'))
            self.Xh = self.Xhs[self.icol]
            self.Q=Q
            return 1
        else:#except:
            return -1
def plot_Xh(self,viewer=None,color='white',dic_th={},rescale=True):
    Xh = self.Xh.copy()
    
    cor = Xh[:,-2]
    h = Xh[:,-1]
    th_br = dic_th.get(self.icol,0)
    Xh = Xh[h>th_br]
    
    h = Xh[:,-1]
    #import napari
    #napari.view_points()
    #bad = cor>0.25
    if rescale:
        min_,max_ = np.sort(h)[np.array([50,-10])%len(h)]
        h_n = np.clip((h-min_)/(max_-min_),0,1)
        size=20*h_n
    else:
        size=10
    import napari
    if viewer is None:
        viewer = napari.Viewer()
    viewer.add_points(Xh[:,:3],size=size,name = self.Q,face_color=color)#,text=h.astype(int).astype(str))
    return viewer

def add_png_image(self,viewer=None,return_im=False):
    import cv2,napari
    self.im_png = im_png = cv2.imread(self.fl.replace('--_Xh_RNAs.pkl','_signal-col'+str(self.icol)+'.png'))
    im_ = im_png[...,0]
    xs = np.where(np.diff(np.percentile(im_,5,axis=1)>254)!=0)[0]+1
    xm,xM = np.min(xs),np.max(xs)
    ys = np.where(np.diff(np.percentile(im_,5,axis=0)>254)!=0)[0]+1
    ym,yM = np.min(ys),np.max(ys)
    im_=im_[xm:xM,ym:yM]
    new_shape = [1]+list(np.max(self.Xh[:,:3],axis=0).astype(int)[1:])
    im__ = resize(im_[np.newaxis],new_shape)
    if return_im:
        return im__
    if viewer is None: viewer = napari.Viewer()
    viewer.add_image(im__)
def get_fov(fl): return os.path.basename(fl).split('--')[0].split('.')[0]
def load_segm(self,force=False,vol_th = 1000,dil=5,stitch_th=0.5):
    self.prev_fov = getattr(self,"prev_fov","")
    self.fov= fov = get_fov(self.fl) 
    if (self.prev_fov!=self.fov) or force or (not hasattr(self,"im_segm")):
        analysis_folder = self.save_folder
        self.prev_fov = fov
        segm_folder = analysis_folder+os.sep+'Segmentation'
        dapi_fls_fl = segm_folder+os.sep+'dapi_fls_fl.npy'
        if not os.path.exists(dapi_fls_fl) or force:
            dapi_fls = glob.glob(segm_folder+os.sep+'*.npz')
            np.save(dapi_fls_fl,dapi_fls)
        else:
            dapi_fls = np.load(dapi_fls_fl)
        self.dapi_fls = dapi_fls
        self.fl_dapi = fl_dapi = [fl for fl in self.dapi_fls if self.fov in fl and self.set_ in get_H_fld(fl)][0]
        #save_fl  = segm_folder+os.sep+os.path.basename(fl_dapi).split('.')[0]+'--'+os.path.basename(os.path.dirname(fl_dapi))+'--dapi_segm.npz'
        self.im_segm,self.shape = np.load(fl_dapi)['segm'],np.load(fl_dapi)['shape']
        self.im_segm = stitch3D(self.im_segm,niter=5,th_int=stitch_th)
        #self.im_segmf = resize(self.im_segm,self.shape)
        import scipy.ndimage as nd
        vols = nd.sum(self.im_segm>-1,self.im_segm,np.unique(self.im_segm))

        bad_cells = np.where(vols<vol_th)[0]
        bad = np.in1d(self.im_segm,bad_cells).reshape(self.im_segm.shape)
        self.im_segm[bad]=0

        self.im_segm_bin = self.im_segm>0
        self.im_segm_bin_dil = nd.binary_dilation(self.im_segm_bin,iterations=dil)
        X_extra = np.array(np.where(self.im_segm_bin_dil>self.im_segm_bin)).T
        X = np.array(np.where(self.im_segm_bin)).T
        from scipy.spatial import KDTree
        Xb = X[KDTree(X).query(X_extra)[1]]
        #self.im_segm_ = self.im_segm.copy()
        self.im_segm[tuple(X_extra.T)] = self.im_segm[tuple(Xb.T)] 
        self.shapesm = self.im_segm.shape
        cells = np.unique(self.im_segm)
        cms = nd.center_of_mass(self.im_segm>-1,self.im_segm,cells)
        self.cms = {c_+self.ifov*10**6:(cm_*self.shape/self.shapesm).astype(int) for c_,cm_ in zip(cells,cms) if c_>0}
def get_counts_per_cell(self,th_cor=0.5):
    dic_th = self.dic_th
    icol = self.icol
    im_segm = self.im_segm
    shapesm = self.im_segm.shape
    shape = self.shape

    Xh = self.Xh.copy()
    cor = Xh[:,-2]
    h = Xh[:,-1]
    keep = h>dic_th.get(icol,0)
    Xh = Xh[keep]
    txyz = np.array(self.dic_drift['txyz'])
    txyz[txyz>0]=0
    Xcms = Xh[:,:3]-txyz
    Xred = np.round((Xcms/shape)*shapesm).astype(int)
    good = ~np.any((Xred>=shapesm)|(Xred<0),axis=-1)
    Xh,Xred = Xh[good],Xred[good]
    self.Xred = Xred
    icells = im_segm[tuple(Xred.T)]
    cells,cts = np.unique(icells[(Xh[:,-2]>th_cor)],return_counts=True)
    self.good_counts = {c_+self.ifov*10**6:ct_ for c_,ct_ in zip(cells,cts) if c_>0}
    cells,cts = np.unique(icells[(Xh[:,-2]<th_cor)],return_counts=True)
    self.bad_counts = {c_+self.ifov*10**6:ct_ for c_,ct_ in zip(cells,cts) if c_>0}
def get_int_im1_im2(im1,im2,th_int=0.5):
    inters = ((im1>0)&(im2>0)).astype(int)
    im1_in2 = im1*inters
    N1max = np.max(im1)+1
    im2_in1 = im2*inters*N1max
    iint,cts = np.unique(im1_in2+im2_in1,return_counts=True)
    c1,cts1 = np.unique(im1,return_counts=True)
    dic_c1 = {c_:ct_ for c_,ct_ in zip(c1,cts1) if c_>0}
    c2,cts2 = np.unique(im2,return_counts=True)
    dic_c2 = {c_:ct_ for c_,ct_ in zip(c2,cts2) if c_>0}
    dic_int= {(c1,c2):(ct/dic_c1[c1],ct/dic_c2[c2]) for c2,c1,ct in zip(iint//N1max,iint%N1max,cts) 
         if (c1>0) and (c2>0) and (c1!=c2)}
    objs1 = nd.find_objects(im1)
    objs2 = nd.find_objects(im2)
    for cch in dic_int:
        c1,c2 = cch
        ic1,ic2 = dic_int[cch]
        obj1,obj2 = objs1[c1-1],objs2[c2-1]
        if (ic1>th_int) or (ic2>th_int):
            c_ = np.min([c1,c2])
            im1[obj1][im1[obj1]==c1]=c_
            im2[obj2][im2[obj2]==c2]=c_
    return im1,im2
def stitch3D(im_segm,niter=5,th_int=0.5):
    for it_ in range(niter):
        for iim in range(len(im_segm)-1):
            im_segm[iim],im_segm[iim+1] = get_int_im1_im2(im_segm[iim],im_segm[iim+1],th_int=th_int)
    return im_segm

In [3]:
dicQ = {'ASCL1': 'Q1', 'Q1': 'ASCL1', 'APLN': 'Q2', 'Q2': 'APLN', 'APOE': 'Q3', 'Q3': 'APOE', 'CSF1R': 'Q4', 'Q4': 'CSF1R', 'ARX': 'Q5', 'Q5': 'ARX', 'AQP4': 'Q6', 'Q6': 'AQP4', 'CSF2RA': 'Q7', 'Q7': 'CSF2RA', 'BGN': 'Q8', 'Q8': 'BGN', 'ARHGEF6': 'Q9', 'Q9': 'ARHGEF6', 'DOCK11': 'Q10', 'Q10': 'DOCK11', 'CFTR': 'Q11', 'Q11': 'CFTR', 'BEX1': 'Q12', 'Q12': 'BEX1', 'FGF10': 'Q13', 'Q13': 'FGF10', 'COL15A1': 'Q14', 'Q14': 'COL15A1', 'BMP2': 'Q15', 'Q15': 'BMP2', 'FLT1': 'Q16', 'Q16': 'FLT1', 'COL4A5': 'Q17', 'Q17': 'COL4A5', 'CD40LG': 'Q18', 'Q18': 'CD40LG', 'FMR1': 'Q19', 'Q19': 'FMR1', 'EDA': 'Q20', 'Q20': 'EDA', 'CD74': 'Q21', 'Q21': 'CD74', 'FNDC1': 'Q22', 'Q22': 'FNDC1', 'EFNB1': 'Q23', 'Q23': 'EFNB1', 'DACH2': 'Q24', 'Q24': 'DACH2', 'FOXP3': 'Q25', 'Q25': 'FOXP3', 'EGFL6': 'Q26', 'Q26': 'EGFL6', 'ENPEP': 'Q27', 'Q27': 'ENPEP', 'GAD1': 'Q28', 'Q28': 'GAD1', 'FGF13': 'Q29', 'Q29': 'FGF13', 'FBN2': 'Q30', 'Q30': 'FBN2', 'IL1RAPL2': 'Q31', 'Q31': 'IL1RAPL2', 'FREM2': 'Q32', 'Q32': 'FREM2', 'FGFR3': 'Q33', 'Q33': 'FGFR3', 'LAMA4': 'Q34', 'Q34': 'LAMA4', 'FRZB': 'Q35', 'Q35': 'FRZB', 'FILIP1L': 'Q36', 'Q36': 'FILIP1L', 'MBP': 'Q37', 'Q37': 'MBP', 'HTR2C': 'Q38', 'Q38': 'HTR2C', 'GFAP': 'Q39', 'Q39': 'GFAP', 'MRC1': 'Q40', 'Q40': 'MRC1', 'IL13RA1': 'Q41', 'Q41': 'IL13RA1', 'IFNG-AS1': 'Q42', 'Q42': 'IFNG-AS1', 'MT-ATP6': 'Q43', 'Q43': 'MT-ATP6', 'LRIG3': 'Q44', 'Q44': 'LRIG3', 'IL3RA': 'Q45', 'Q45': 'IL3RA', 'MT-ND3': 'Q46', 'Q46': 'MT-ND3', 'MAOA': 'Q47', 'Q47': 'MAOA', 'ITM2A': 'Q48', 'Q48': 'ITM2A', 'MT-RNR1': 'Q49', 'Q49': 'MT-RNR1', 'MT-ND1': 'Q50', 'Q50': 'MT-ND1', 'KLHL1': 'Q51', 'Q51': 'KLHL1', 'NPY': 'Q52', 'Q52': 'NPY', 'MT-ND4': 'Q53', 'Q53': 'MT-ND4', 'LAMP5': 'Q54', 'Q54': 'LAMP5', 'PAX6': 'Q55', 'Q55': 'PAX6', 'MT-RNR2': 'Q56', 'Q56': 'MT-RNR2', 'MT-ND2': 'Q57', 'Q57': 'MT-ND2', 'PCDH11X': 'Q58', 'Q58': 'PCDH11X', 'NRGN': 'Q59', 'Q59': 'NRGN', 'MT-ND5': 'Q60', 'Q60': 'MT-ND5', 'PLP1': 'Q61', 'Q61': 'PLP1', 'OLIG1': 'Q62', 'Q62': 'OLIG1', 'NDP': 'Q63', 'Q63': 'NDP', 'PTCHD1-AS': 'Q64', 'Q64': 'PTCHD1-AS', 'OPALIN': 'Q65', 'Q65': 'OPALIN', 'ROBO1': 'Q66', 'Q66': 'ROBO1', 'PVALB': 'Q67', 'Q67': 'PVALB', 'OPHN1': 'Q68', 'Q68': 'OPHN1', 'SYTL5': 'Q69', 'Q69': 'SYTL5', 'SMYD1': 'Q70', 'Q70': 'SMYD1', 'PDGFRA': 'Q71', 'Q71': 'PDGFRA', 'TAC3': 'Q72', 'Q72': 'TAC3', 'SST': 'Q73', 'Q73': 'SST', 'RORB': 'Q74', 'Q74': 'RORB', 'TENM1': 'Q75', 'Q75': 'TENM1', 'TRPC5': 'Q76', 'Q76': 'TRPC5', 'RPRM': 'Q77', 'Q77': 'RPRM', 'TSIX': 'Q78', 'Q78': 'TSIX', '': 'Q114', 'Q79': '', 'RTKN2': 'Q80', 'Q80': 'RTKN2', 'TYROBP': 'Q81', 'Q81': 'TYROBP', 'Q82': '', 'SCML4': 'Q83', 'Q83': 'SCML4', 'VCAN': 'Q84', 'Q84': 'VCAN', 'Q85': '', 'SMOC1': 'Q86', 'Q86': 'SMOC1', 'VIP': 'Q87', 'Q87': 'VIP', 'Q88': '', 'SRGN': 'Q89', 'Q89': 'SRGN', 'Xist': 'Q90', 'Q90': 'Xist', 'Q91': '', 'SRPX2': 'Q92', 'Q92': 'SRPX2', 'Q93': '', 'Q94': '', 'TEX13A': 'Q95', 'Q95': 'TEX13A', 'Q96': '', 'Q97': '', 'TH': 'Q98', 'Q98': 'TH', 'Q99': '', 'Q100': '', 'THEMIS': 'Q101', 'Q101': 'THEMIS', 'Q102': '', 'Q103': '', 'TIMP1': 'Q104', 'Q104': 'TIMP1', 'Q105': '', 'Q106': '', 'TMSB4X': 'Q107', 'Q107': 'TMSB4X', 'Q108': '', 'Q109': '', 'TNNT2': 'Q110', 'Q110': 'TNNT2', 'Q111': '', 'Q112': '', 'WNT4': 'Q113', 'Q113': 'WNT4', 'Q114': ''}

### Visualize raw max projection data

In [18]:
import napari
viewer2 = napari.Viewer()

In [5]:
pa = post_analysis(r'Y:\Glass_MERFISH\CGBB_ChrX_01_31_2023_Analysis\*_AD*_Xh_RNAs.pkl',force=False)

In [6]:
dic_im = {}

In [7]:
np.sort([gn for gn in dicQ if len(gn) if gn[0]!='Q'])

array(['APLN', 'APOE', 'AQP4', 'ARHGEF6', 'ARX', 'ASCL1', 'BEX1', 'BGN',
       'BMP2', 'CD40LG', 'CD74', 'CFTR', 'COL15A1', 'COL4A5', 'CSF1R',
       'CSF2RA', 'DACH2', 'DOCK11', 'EDA', 'EFNB1', 'EGFL6', 'ENPEP',
       'FBN2', 'FGF10', 'FGF13', 'FGFR3', 'FILIP1L', 'FLT1', 'FMR1',
       'FNDC1', 'FOXP3', 'FREM2', 'FRZB', 'GAD1', 'GFAP', 'HTR2C',
       'IFNG-AS1', 'IL13RA1', 'IL1RAPL2', 'IL3RA', 'ITM2A', 'KLHL1',
       'LAMA4', 'LAMP5', 'LRIG3', 'MAOA', 'MBP', 'MRC1', 'MT-ATP6',
       'MT-ND1', 'MT-ND2', 'MT-ND3', 'MT-ND4', 'MT-ND5', 'MT-RNR1',
       'MT-RNR2', 'NDP', 'NPY', 'NRGN', 'OLIG1', 'OPALIN', 'OPHN1',
       'PAX6', 'PCDH11X', 'PDGFRA', 'PLP1', 'PTCHD1-AS', 'PVALB', 'ROBO1',
       'RORB', 'RPRM', 'RTKN2', 'SCML4', 'SMOC1', 'SMYD1', 'SRGN',
       'SRPX2', 'SST', 'SYTL5', 'TAC3', 'TENM1', 'TEX13A', 'TH', 'THEMIS',
       'TIMP1', 'TMSB4X', 'TNNT2', 'TRPC5', 'TSIX', 'TYROBP', 'VCAN',
       'VIP', 'WNT4', 'Xist'], dtype='<U9')

In [23]:
gene = 'MT-ND3'
Q = dicQ[gene]
#pa = post_analysis(r'Y:\Glass_MERFISH\CGBB_ChrX_01_31_2023_Analysis\*_AD*_Xh_RNAs.pkl')
pa.get_Xh_Q(ifov=67,Q = Q)#30,80!
ims = np.load(pa.fl.replace('--_Xh_RNAs.pkl','_plot_ims.npz'))["plot_ims"]
dic_im[gene] = ims[pa.icol]#add_png_image(pa,viewer=viewr,return_im=True)

viewer2.add_image(dic_im[gene],name=gene,contrast_limits=[0,20000],blending='additive')

<Image layer 'MT-ND3' at 0x2600074f790>

In [20]:
plot_Xh(pa,viewer=viewer2,color='red',dic_th={0:5000,1:5000,2:5000},rescale=False);
plot_Xh(pa,viewer=viewer2,color='green',dic_th={0:3000,1:3000,2:3000},rescale=False);

In [46]:
gene = 'GAD1'
Q = dicQ[gene]
#pa = post_analysis(r'Y:\Glass_MERFISH\CGBB_ChrX_01_31_2023_Analysis\*_AD*_Xh_RNAs.pkl')
pa.get_Xh_Q(ifov=80,Q = Q)
plot_Xh(pa,viewer=viewer2,color='white',dic_th={0:5000,1:5000,2:5000},rescale=False);

In [ ]:
viewer2

In [145]:
dic_im = {}

In [641]:

for gene in ["LAMP5",'SST','PVALB','RORB','ARX','CSF1R']:
    Q = dicQ[gene]
    pa = post_analysis(r'Y:\Glass_MERFISH\CGBB_ChrX_01_31_2023_Analysis\*_C*_Xh_RNAs.pkl')
    pa.get_Xh_Q(ifov=70,Q = Q)
    ims = np.load(pa.fl.replace('--_Xh_RNAs.pkl','_plot_ims.npz'))["plot_ims"]
    dic_im[gene] = ims[pa.icol]#add_png_image(pa,viewer=viewr,return_im=True)

    viewer.add_image(dic_im[gene],name=gene,contrast_limits=[0,20000],blending='additive')

In [173]:
mtgns = [gn for gn in dicQ if'MT' in gn]

In [25]:
gene = 'NRGN'
for gene in mtgns:
    Q = dicQ[gene]
    pa = post_analysis(r'Y:\Glass_MERFISH\CGBB_ChrX_01_31_2023_Analysis\*_C*_Xh_RNAs.pkl')
    pa.get_Xh_Q(ifov=70,Q = Q)
    ims = np.load(pa.fl.replace('--_Xh_RNAs.pkl','_plot_ims.npz'))["plot_ims"]
    dic_im[gene] = ims[pa.icol]#add_png_image(pa,viewer=viewr,return_im=True)

    viewer.add_image(dic_im[gene],name=gene,contrast_limits=[0,20000],blending='additive')

NameError: name 'mtgns' is not defined

In [41]:
dapi_fls = np.load(pa.save_folder+os.sep+'files_map.npz')['dapi_fls']

In [42]:
pa.set_ = '_AD'
fl = [fl for fl in dapi_fls if pa.set_ in os.path.basename(fl) and get_ifov(fl)==pa.ifov][0]
im_seg =np.load(fl)['segm']
shape = np.load(fl)['shape']
im_seg_ = resize(im_seg,shape)

viewer.add_image(np.mean(im_seg_>0,axis=0),name="DAPI_seg",contrast_limits=[0,1],blending='additive')

<Image layer 'DAPI_seg' at 0x24909230dc0>

In [43]:
viewer2.add_image(np.mean(im_seg_>0,axis=0),name="DAPI_seg",contrast_limits=[0,1],blending='additive')

<Image layer 'DAPI_seg' at 0x2490927e2e0>

In [589]:
all_genes = [e for e in dicQ if e[:1]!='Q' and len(e)]

In [625]:
dic_ct_fov = {}
pa = post_analysis(r'Y:\Glass_MERFISH\CGBB_ChrX_01_31_2023_Analysis\*_C*_Xh_RNAs.pkl',force=True)

In [617]:
os.path.isdir

<function genericpath.isdir(s)>

In [141]:
gene = "AQP4"
Q = dicQ[gene]
pa = post_analysis(r'Y:\Glass_MERFISH\CGBB_ChrX_01_31_2023_Analysis\*_C*_Xh_RNAs.pkl')
pa.get_Xh_Q(ifov=70,Q = Q)
viewr = plot_Xh(pa,viewer=None,color='white',dic_th={},rescale=True)

In [656]:
gene = "PVALB"
Q = dicQ[gene]
pa = post_analysis(r'Y:\Glass_MERFISH\CGBB_ChrX_01_31_2023_Analysis\*_C*_Xh_RNAs.pkl')
pa.get_Xh_Q(ifov=20,Q = Q)
V =plot_Xh(pa,viewer=None,color='white',dic_th={},rescale=True)

In [658]:
plot_Xh_bad(pa,viewer=V,color='red',dic_th={},rescale=True,cor_th =0.5)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 1433.4943149737364, 1440.016694386004), zoom=0.33892622438633124, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1.0, 1.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=3, ndisplay=2, last_used=0, range=((-0.7099205007777517, 59.569249679874, 1.0), (-0.18699231357237842, 2868.175622261045, 1.0), (0.004014247857868755, 2881.02937452415, 1.0)), current_step=(30, 1433, 1440), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Points layer 'Q67' at 0x25fdef8f580>, <Points layer 'Q67 [1]' at 0x25fcad04f40>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10, unit=None), text_overlay=TextOverlay(visible=False, color=(0.5, 0.5, 0.5, 1.0), font_size=10, position=<TextOver

In [657]:
def plot_Xh_bad(self,viewer=None,color='white',dic_th={},rescale=True,cor_th =0.5):
    Xh = self.Xh.copy()
    
    cor = Xh[:,-2]
    Xh = Xh[cor<cor_th]
    
    h = Xh[:,-1]
    #import napari
    #napari.view_points()
    #bad = cor>0.25
    if rescale:
        min_,max_ = np.sort(h)[np.array([50,-10])%len(h)]
        h_n = np.clip((h-min_)/(max_-min_),0,1)
        size=20*h_n
    else:
        size=10
    import napari
    if viewer is None:
        viewer = napari.Viewer()
    viewer.add_points(Xh[:,:3],size=size,name = self.Q,face_color=color)#,text=h.astype(int).astype(str))
    return viewer

In [ ]:
plot_Xh(pa,viewer=None,color='white',dic_th={},rescale=True)

In [650]:
pa.set_ = '_C'
load_segm(pa,force=False,vol_th = 1500,dil=1)

In [653]:
im_segm = resize(pa.im_segm,pa.shape)


In [654]:
viewr.add_labels(im_segm)------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

<Labels layer 'im_segm' at 0x25f8c965790>

### Main post analysis

In [49]:
all_genes=np.sort([gn for gn in dicQ if len(gn) if gn[0]!='Q'])

In [60]:
dic_ct_fov = {}
ifov=80
pa.set_ = '_AD'
save_fld = pa.save_folder+os.sep+'dic_cts'
if not os.path.exists(save_fld): os.makedirs(save_fld)
save_fl = save_fld+os.sep+str(ifov).zfill(5)+'--'+pa.set_+'--dic_cts.pkl'
if not os.path.exists(save_fl):
    for gene in tqdm(all_genes):
        #gene = "PVALB"

        Q = dicQ[gene]
        pa.get_Xh_Q(ifov=ifov,Q = Q) #load in the points
        load_segm(pa,force=False,vol_th = 1500,dil=5) #load in the segmentation file
        pa.dic_th={0:5000,1:5000,2:5000}
        get_counts_per_cell(pa)
        dic_ct_fov[gene]=(pa.good_counts,pa.bad_counts)


    print(save_fl)
    pickle.dump([dic_ct_fov,pa.cms],open(save_fl,'wb'))

100%|██████████████████████████████████████████████████████████████████████████████████| 94/94 [00:10<00:00,  9.16it/s]

Y:\Glass_MERFISH\CGBB_ChrX_01_31_2023_Analysis\dic_cts\00080--_AD--dic_cts.pkl


In [ ]:
#gene = "PVALB"
nfovs = len(pa.dic_fls.keys())
for ifov in range(nfovs):
    try:
        dic_ct_fov = {}
        pa.set_ = '_AD'
        save_fld = pa.save_folder+os.sep+'dic_cts'
        if not os.path.exists(save_fld): os.makedirs(save_fld)
        save_fl = save_fld+os.sep+str(ifov).zfill(5)+'--'+pa.set_+'--_3000-dic_cts.pkl'
        if not os.path.exists(save_fl):
            for gene in tqdm(all_genes):
                #gene = "PVALB"

                Q = dicQ[gene]
                pa.get_Xh_Q(ifov=ifov,Q = Q) #load in the points
                load_segm(pa,force=False,vol_th = 1500,dil=5) #load in the segmentation file
                pa.dic_th={0:3000,1:3000,2:3000}
                get_counts_per_cell(pa)
                dic_ct_fov[gene]=(pa.good_counts,pa.bad_counts)


            print(save_fl)
            pickle.dump([dic_ct_fov,pa.cms],open(save_fl,'wb'))
    except:
        print("Failed************************** "+str(ifov))

100%|██████████████████████████████████████████████████████████████████████████████████| 94/94 [00:37<00:00,  2.51it/s]


Y:\Glass_MERFISH\CGBB_ChrX_01_31_2023_Analysis\dic_cts\00000--_AD--_3000-dic_cts.pkl


100%|██████████████████████████████████████████████████████████████████████████████████| 94/94 [00:22<00:00,  4.14it/s]


Y:\Glass_MERFISH\CGBB_ChrX_01_31_2023_Analysis\dic_cts\00001--_AD--_3000-dic_cts.pkl


100%|██████████████████████████████████████████████████████████████████████████████████| 94/94 [00:49<00:00,  1.90it/s]


...

100%|██████████████████████████████████████████████████████████████████████████████████| 94/94 [00:27<00:00,  3.37it/s]


Y:\Glass_MERFISH\CGBB_ChrX_01_31_2023_Analysis\dic_cts\00056--_AD--_3000-dic_cts.pkl


100%|██████████████████████████████████████████████████████████████████████████████████| 94/94 [00:20<00:00,  4.55it/s]


Y:\Glass_MERFISH\CGBB_ChrX_01_31_2023_Analysis\dic_cts\00057--_AD--_3000-dic_cts.pkl


 67%|██████████████████████████████████████████████████████▉                           | 63/94 [00:13<00:03,  8.20it/s]

In [628]:
#pa.dic_fls[pa.ifov]

Y:\Glass_MERFISH\CGBB_ChrX_01_31_2023_Analysis\dic_cts\Conv_zscan__020--_C--dic_cts.pkl


In [593]:
self.fov

'Conv_zscan__020'

In [518]:
#pa.dic_th={0:2000,1:2000,2:2000}
self=pa

    

In [531]:
pa.good_counts,pa.bad_counts

({9: 1,
  19: 1,
  22: 56,
  28: 64,
  30: 1,
  44: 1,
  60: 19,
  67: 6,
  68: 1,
  73: 1,
  75: 1,
  77: 1,
  121: 1,
  126: 7},
 {16: 1, 28: 100, 45: 2, 67: 23, 126: 10})

In [532]:
viewer = napari.Viewer()
viewer.add_labels(pa.im_segm)
viewer.add_points(pa.Xred,size=2)

<Points layer 'Points' at 0x25f941cebb0>

In [498]:

self=pa
self.dic_drift['txyz']

array([ 1, 32, 19])

In [468]:
self.shape

array([  60, 2900, 2900])

In [460]:
np.mean(cor[h>dic_th.get(icol,0)]>0.5)

0.3881118881118881

In [458]:
np.mean(cor[h>dic_th.get(icol,0)]>0.5)

0.7563261480787254

In [457]:
plot_Xh(pa,viewer=None,color='white',dic_th={},rescale=True)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 1433.4943149737364, 1440.016694386004), zoom=0.1769221644382985, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1.0, 1.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=3, ndisplay=2, last_used=0, range=((-0.7099205007777517, 59.569249679874, 1.0), (-0.18699231357237842, 2868.175622261045, 1.0), (0.004014247857868755, 2881.02937452415, 1.0)), current_step=(30, 1433, 1440), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Points layer 'Q67' at 0x25f0fae8af0>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10, unit=None), text_overlay=TextOverlay(visible=False, color=(0.5, 0.5, 0.5, 1.0), font_size=10, position=<TextOverlayPosition.TOP_LEFT: 'top_left'>, text=''),

In [411]:
viewer = napari.view_labels(pa.im_segm)
#viewer.add_labels(im_segm)

<Labels layer 'im_segm' at 0x2544e226df0>

In [418]:

#viewer = napari.view_labels(pa.im_segm)

In [420]:
def get_counts_per_cell(self,nbad=0):
    keep_good = np.sum(self.res_pruned==-1,axis=-1)<=nbad
    Xcms = self.Xcms[keep_good]
    icodes = self.icodes[keep_good]
    Xred = np.round((Xcms/self.shape)*self.shapesm).astype(int)
    good = ~np.any((Xred>=self.shapesm)|(Xred<0),axis=-1)
    Xred = Xred[good]
    icodes = icodes[good]
    cts_all = []
    for ikeep in np.arange(len(self.gns_names)):
        Xred_ = Xred[icodes==ikeep]
        icells,cts = np.unique(self.im_segm[tuple(Xred_.T)],return_counts=True)
        dic_cts = {icell:ct for icell,ct in zip(icells,cts)}
        ctsf = [dic_cts.get(icell,0) for icell in self.icells]
        cts_all.append(ctsf)
    cts_all = np.array(cts_all)
    return cts_all

In [309]:
from cellpose.utils import stitch3D

In [315]:
pa.get_Xh_Q(ifov=11,Q = Q)

1

In [419]:
load_segm(pa,force=False,vol_th = 1500,dil=5)


In [314]:
viewer = napari.view_labels(pa.im_segm)
viewer.add_labels(stitch3D(pa.im_segm,stitch_threshold=0.01))

C:\Users\BintuLabUser\anaconda3\envs\cellpose\lib\site-packages\cellpose\metrics.py:202: RuntimeWarning: invalid value encountered in true_divide
  iou = overlap / (n_pixels_pred + n_pixels_true - overlap)


<Labels layer 'Labels [1]' at 0x2544dd55eb0>

In [311]:
np.any(stitch3D(pa.im_segm)!=pa.im_segm)

C:\Users\BintuLabUser\anaconda3\envs\cellpose\lib\site-packages\cellpose\metrics.py:202: RuntimeWarning: invalid value encountered in true_divide
  iou = overlap / (n_pixels_pred + n_pixels_true - overlap)


False

In [312]:
??stitch3D

In [302]:
self.prev_fov,self.fov

('Conv_zscan__070', 'Conv_zscan__070')

In [276]:
viewer = napari.view_labels(pa.im_segm)
viewer.add_labels(pa.im_segm_)

<Labels layer 'Labels [1]' at 0x253eeb31f40>

  File "C:\Users\BintuLabUser\anaconda3\envs\cellpose\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\BintuLabUser\anaconda3\envs\cellpose\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\BintuLabUser\anaconda3\envs\cellpose\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\BintuLabUser\anaconda3\envs\cellpose\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
    app.start()
  File "C:\Users\BintuLabUser\anaconda3\envs\cellpose\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
    self.io_loop.start()
  File "C:\Users\BintuLabUser\anaconda3\envs\cellpose\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
    self.asyncio_loop.run_forever()
  File "C:\Users\BintuLabUser\anaconda3\envs\cellpose\lib\asyncio\base_events.py", line 596, in run_forever
    self._run_once()
  File 

<Labels layer 'Labels [2]' at 0x253dfa978e0>

In [218]:
pa.set_='_C'

In [ ]:
pa.set_='_C'

In [210]:
pa.set_

AttributeError: 'post_analysis' object has no attribute 'set_'

In [209]:

def get_H_fld(fl):return os.path.basename(fl).split('--')[1]

In [208]:
#[for fl in dapi_fls if ]
#fov = get_fov(pa.fl)
#get_(pa.fl)
??get_H

In [195]:
self = pa
analysis_folder = self.save_folder
segm_folder = analysis_folder+os.sep+'Segmentation'
dapi_fls = glob.glob(segm_folder+os.sep+'*.npz')

In [202]:
get_fov(self.fl)

'Conv_zscan__070'

In [192]:
pa.save_folder

'Y:\\Glass_MERFISH\\CGBB_ChrX_01_31_2023_Analysis'

In [ ]:
np.load(self.base_save+'_plot_ims.npz',plot_ims = self.plot_ims)

In [128]:
dic_im = {}

In [189]:
viewer = napari.Viewer()

In [178]:
print(dic_Q)

NameError: name 'dic_Q' is not defined

In [180]:
print(dicQ)

{'ASCL1': 'Q1', 'Q1': 'ASCL1', 'APLN': 'Q2', 'Q2': 'APLN', 'APOE': 'Q3', 'Q3': 'APOE', 'CSF1R': 'Q4', 'Q4': 'CSF1R', 'ARX': 'Q5', 'Q5': 'ARX', 'AQP4': 'Q6', 'Q6': 'AQP4', 'CSF2RA': 'Q7', 'Q7': 'CSF2RA', 'BGN': 'Q8', 'Q8': 'BGN', 'ARHGEF6': 'Q9', 'Q9': 'ARHGEF6', 'DOCK11': 'Q10', 'Q10': 'DOCK11', 'CFTR': 'Q11', 'Q11': 'CFTR', 'BEX1': 'Q12', 'Q12': 'BEX1', 'FGF10': 'Q13', 'Q13': 'FGF10', 'COL15A1': 'Q14', 'Q14': 'COL15A1', 'BMP2': 'Q15', 'Q15': 'BMP2', 'FLT1': 'Q16', 'Q16': 'FLT1', 'COL4A5': 'Q17', 'Q17': 'COL4A5', 'CD40LG': 'Q18', 'Q18': 'CD40LG', 'FMR1': 'Q19', 'Q19': 'FMR1', 'EDA': 'Q20', 'Q20': 'EDA', 'CD74': 'Q21', 'Q21': 'CD74', 'FNDC1': 'Q22', 'Q22': 'FNDC1', 'EFNB1': 'Q23', 'Q23': 'EFNB1', 'DACH2': 'Q24', 'Q24': 'DACH2', 'FOXP3': 'Q25', 'Q25': 'FOXP3', 'EGFL6': 'Q26', 'Q26': 'EGFL6', 'ENPEP': 'Q27', 'Q27': 'ENPEP', 'GAD1': 'Q28', 'Q28': 'GAD1', 'FGF13': 'Q29', 'Q29': 'FGF13', 'FBN2': 'Q30', 'Q30': 'FBN2', 'IL1RAPL2': 'Q31', 'Q31': 'IL1RAPL2', 'FREM2': 'Q32', 'Q32': 'FREM2', 'FGFR

<Image layer 'PDGFRA' at 0x253cc7a3520>

In [165]:
asm = analysis_smFISH(data_folders = [r'Y:\Glass_MERFISH\CGBB_ChrX_01_31_2023'],
                     save_folder =r'Y:\Glass_MERFISH\CGBB_ChrX_01_31_2023_Analysis',
                     H0folder=  r'Y:\Glass_MERFISH\CGBB_ChrX_01_31_2023\H0_*',exclude_H0=True)

Found files:309
Found hybe folders:194


In [173]:
set_,ifov = '_C',70
asm.set_set(set_)
Qfolders_keep = [e for e in asm.Qfolders if asm.set_ in os.path.basename(e)]

iQ = [iQ for iQ,Qfld in enumerate(Qfolders_keep) 
 if 'Lamin_C2' in os.path.basename(Qfld)][0]
asm.set_fov(ifov)
asm.set_hybe(iQ)

In [174]:
asm.get_background(force=False)
asm.get_signal()
asm.compute_drift(sz=300)
asm.get_aligned_ims()

### define H0 and load background
Background file: Y:\Glass_MERFISH\CGBB_ChrX_01_31_2023\H0_C\Conv_zscan__070.zarr
### load signal
Signal file: Y:\Glass_MERFISH\CGBB_ChrX_01_31_2023\H68_Lamin_C2\Conv_zscan__070.zarr
Found drift:[  4  59 -19]


In [176]:
imsg = asm.im_sig__[0].astype(np.float32)

In [177]:
viewer.add_image(imsg,name='LaminA')

<Image layer 'LaminA' at 0x253d17690d0>

  File "C:\Users\BintuLabUser\anaconda3\envs\cellpose\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\BintuLabUser\anaconda3\envs\cellpose\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\BintuLabUser\anaconda3\envs\cellpose\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\BintuLabUser\anaconda3\envs\cellpose\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
    app.start()
  File "C:\Users\BintuLabUser\anaconda3\envs\cellpose\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
    self.io_loop.start()
  File "C:\Users\BintuLabUser\anaconda3\envs\cellpose\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
    self.asyncio_loop.run_forever()
  File "C:\Users\BintuLabUser\anaconda3\envs\cellpose\lib\asyncio\base_events.py", line 596, in run_forever
    self._run_once()
  File 

In [155]:
for layer in viewer.layers:
    layer.data

[<Image layer 'AQP4' at 0x2547ad9efd0>, <Image layer 'PLP1' at 0x253cd7bee20>, <Image layer 'RORB' at 0x253cc626340>, <Image layer 'GFAP' at 0x253cc9019d0>, <Image layer 'GAD1' at 0x253cd36d1f0>, <Image layer 'FLT1' at 0x253cc7f44c0>]

In [149]:

print(dicQ)

{'ASCL1': 'Q1', 'Q1': 'ASCL1', 'APLN': 'Q2', 'Q2': 'APLN', 'APOE': 'Q3', 'Q3': 'APOE', 'CSF1R': 'Q4', 'Q4': 'CSF1R', 'ARX': 'Q5', 'Q5': 'ARX', 'AQP4': 'Q6', 'Q6': 'AQP4', 'CSF2RA': 'Q7', 'Q7': 'CSF2RA', 'BGN': 'Q8', 'Q8': 'BGN', 'ARHGEF6': 'Q9', 'Q9': 'ARHGEF6', 'DOCK11': 'Q10', 'Q10': 'DOCK11', 'CFTR': 'Q11', 'Q11': 'CFTR', 'BEX1': 'Q12', 'Q12': 'BEX1', 'FGF10': 'Q13', 'Q13': 'FGF10', 'COL15A1': 'Q14', 'Q14': 'COL15A1', 'BMP2': 'Q15', 'Q15': 'BMP2', 'FLT1': 'Q16', 'Q16': 'FLT1', 'COL4A5': 'Q17', 'Q17': 'COL4A5', 'CD40LG': 'Q18', 'Q18': 'CD40LG', 'FMR1': 'Q19', 'Q19': 'FMR1', 'EDA': 'Q20', 'Q20': 'EDA', 'CD74': 'Q21', 'Q21': 'CD74', 'FNDC1': 'Q22', 'Q22': 'FNDC1', 'EFNB1': 'Q23', 'Q23': 'EFNB1', 'DACH2': 'Q24', 'Q24': 'DACH2', 'FOXP3': 'Q25', 'Q25': 'FOXP3', 'EGFL6': 'Q26', 'Q26': 'EGFL6', 'ENPEP': 'Q27', 'Q27': 'ENPEP', 'GAD1': 'Q28', 'Q28': 'GAD1', 'FGF13': 'Q29', 'Q29': 'FGF13', 'FBN2': 'Q30', 'Q30': 'FBN2', 'IL1RAPL2': 'Q31', 'Q31': 'IL1RAPL2', 'FREM2': 'Q32', 'Q32': 'FREM2', 'FGFR

In [187]:
Q = dicQ["NRGN"]
pa = post_analysis(r'Y:\Glass_MERFISH\CGBB_ChrX_01_31_2023_Analysis\*_C*_Xh_RNAs.pkl')
get_Xh_Q(pa,ifov=70,Q = Q)
viewr = plot_Xh(pa,viewer=None,color='green',dic_th={0:5000,1:5000,2:5000},rescale=False);
plot_Xh(pa,viewer=viewr,color='red')
add_png_image(pa,viewer=viewr)

In [121]:
pa = post_analysis(r'Y:\Glass_MERFISH\CGBB_ChrX_01_31_2023_Analysis\*_C*_Xh_RNAs.pkl')

In [123]:
get_Xh_Q(pa,ifov=20,Q = Q)
add_png_image(pa,viewer=None)

In [124]:
pa.icol

0

In [76]:
[1,list(np.max(pa.Xh[:,:3],axis=0).astype(int)[1:])]

[2895, 2899]

In [26]:
 np.where(np.diff(np.mean(im_,axis=1)>254)!=0)[0]

array([ 478, 3561, 3570, 3580], dtype=int64)

In [13]:
import napari
napari.view_image(im_png)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 1999.5, 1999.5), zoom=0.30708749999999996, angles=(0.0, 0.0, 90.0), perspective=0, interactive=True), cursor=Cursor(position=(1, 1), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=2, ndisplay=2, last_used=0, range=((0.0, 4000.0, 1.0), (0.0, 4000.0, 1.0)), current_step=(2000, 2000), order=(0, 1), axis_labels=('0', '1')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'im_png' at 0x253888e5670>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10, unit=None), text_overlay=TextOverlay(visible=False, color=(0.5, 0.5, 0.5, 1.0), font_size=10, position=<TextOverlayPosition.TOP_LEFT: 'top_left'>, text=''), overlays=Overlays(interaction_box=InteractionBox(points=None, show=False, show_handle=False, show_vertices=False, selection_box_drag=None, sel

In [104]:
np.median(pa.Xh[:,-1])+np.std(pa.Xh[:,-1])*5

4304.580852331805

In [98]:
np.round(np.sort(pa.Xh[:,-1])[-400:])

array([ 9916.,  9930.,  9944.,  9951.,  9956.,  9985.,  9994., 10010.,
       10020., 10051., 10077., 10094., 10116., 10119., 10127., 10133.,
       10143., 10148., 10184., 10187., 10193., 10199., 10203., 10211.,
       10219., 10220., 10228., 10234., 10268., 10281., 10284., 10291.,
       10291., 10302., 10329., 10332., 10340., 10353., 10365., 10405.,
       10428., 10434., 10440., 10451., 10499., 10508., 10516., 10576.,
       10580., 10582., 10586., 10593., 10606., 10623., 10624., 10644.,
       10647., 10653., 10667., 10680., 10710., 10718., 10725., 10728.,
       10753., 10759., 10785., 10788., 10823., 10831., 10839., 10860.,
       10862., 10873., 10875., 10902., 10913., 10981., 10988., 10989.,
       10990., 10990., 11007., 11040., 11042., 11045., 11056., 11069.,
       11076., 11081., 11087., 11107., 11123., 11131., 11185., 11208.,
       11217., 11258., 11291., 11322., 11325., 11336., 11349., 11363.,
       11375., 11382., 11396., 11420., 11447., 11479., 11503., 11504.,
      

In [88]:
get_Xh_Q(pa,ifov=40,Q = 'Q4')
viewer = plot_Xh(pa,viewer=None,color='green')
get_Xh_Q(pa,ifov=40,Q = 'Q5')
plot_Xh(pa,viewer=viewer,color='cyan');
get_Xh_Q(pa,ifov=40,Q = 'Q6')
plot_Xh(pa,viewer=viewer,color='red');

In [37]:
get_Xh_Q(pa,ifov=20,Q = 'Q4')

1

In [29]:
pa.icol

0

In [ ]:
cor = Xh[:,-2]
h = Xh[:,-1]
#import napari
#napari.view_points()
bad = cor<0.5
min_,max_ = np.sort(h)[np.array([50,-10])%len(h)]
h_n = np.clip((h-min_)/(max_-min_),0,1)
size=20*h_n
import napari
napari.view_points(Xh[:,:3],size=size)
